## Sistemas de Recomendación - Parte II
En esta actividad vamos a desarrollar diferentes algoritmos de recomendación para aprender su implementación en Python utilizando diferentes librerías.

Para ello utilizaremos el dataset de MovieLens el cual es una colección de datos ampliamente utilizada en el área de sistemas de recomendación y se centra en la recomendación de películas. Fue creado originalmente por el Grupo de Investigación de Sistemas de Recomendación de GroupLens y ha sido utilizado en investigaciones y competiciones relacionadas con sistemas de recomendación.

El dataset de Movielens contiene calificaciones y reseñas de usuarios sobre películas. Los datos incluyen información sobre películas, como su título, género y etiquetas, así como información sobre los usuarios, como su identificación y características demográficas.

En general, hay varios tamaños de datasets disponibles, que varían en la cantidad de calificaciones y la cantidad de usuarios y películas incluidas. Los datasets más comunes son el "Movielens 100k", el "Movielens 1M" y el "Movielens 10M", que contienen aproximadamente 100,000, 1 millón y 10 millones de calificaciones, respectivamente.

Estos datos son ampliamente utilizados para entrenar y evaluar algoritmos de recomendación, como filtrado colaborativo, filtrado basado en contenido y técnicas de factorización de matrices, con el objetivo de generar recomendaciones personalizadas y precisas para los usuarios en función de sus preferencias históricas y las calificaciones de otras películas.

**En esta actividad desarrollaremos dos algoritmos de recomendación vistos en este modulo:**

1. Filtrado basado en contenido
2. Factorización de Matrices

El objetivo es que los alumnos aprendan a desarrollar estas tecnicas de recomendacion en Python.

### Filtrado basado en contenido

En este ejemplo, utilizaremos el dataset MovieLens 100k, que contiene 100,000 calificaciones de películas proporcionadas por usuarios. Para ello debemos descargar  y descomprimir (en el mismo directorio que esta actividad) el dataset que se encuentra en el sitio de MovieLens bajo el siguiente titulo: [MovieLens 100K Dataset](https://grouplens.org/datasets/movielens/)

Para desarrollar el algoritmo de filtrado basado en contenido vamos a utilizar la técnica de TF-IDF para representar el contenido de las películas y luego calculamos la similitud de coseno entre las películas basada en estas representaciones.

Primero, vamos a importar todas las librerías necesarias y luego cargamos el dataset (ratings y peliculas)

In [1]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity


# Cargar el archivo de calificaciones y películas
ratings = pd.read_csv('ml-100k/u.data', sep='\t', names=['user_id', 'movie_id', 'rating', 'timestamp'])
movies = pd.read_csv('ml-100k/u.item', sep='|', names=['movie_id', 'title', 'release_date', 'video_release_date', 'imdb_url',
                                                       'unknown', 'action', 'adventure', 'animation', 'children', 'comedy',
                                                       'crime', 'documentary', 'drama', 'fantasy', 'film_noir', 'horror',
                                                       'musical', 'mystery', 'romance', 'sci_fi', 'thriller', 'war', 'western'],
                     encoding='latin-1')

In [2]:
ratings.head()

,user_id,movie_id,rating,timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596


In [3]:
movies.head()

,movie_id,title,release_date,video_release_date,imdb_url,unknown,action,adventure,animation,children,...,fantasy,film_noir,horror,musical,mystery,romance,sci_fi,thriller,war,western
0,1,Toy Story (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Toy%20Story%2...,0,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0
1,2,GoldenEye (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?GoldenEye%20(...,0,1,1,0,0,...,0,0,0,0,0,0,0,1,0,0
2,3,Four Rooms (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Four%20Rooms%...,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,4,Get Shorty (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Get%20Shorty%...,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5,Copycat (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Copycat%20(1995),0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0


Para utilizar TF-IDF (Term Frequency-Inverse Document Frequency) en el filtrado basado en contenido con el dataset MovieLens, necesitaremos realizar algunos pasos adicionales para procesar los datos y calcular las características TF-IDF de cada película. 

En particular, los generos de las peliculas en este dataset estan representados como columnas separadas (dentro del dataset) con valores de 1 y 0 (one-hot encoding). 

Para usar TF-IDF lo que tenemos que hacer es juntar esta informacion en una única columna que contenga la lista de todos los generos.

In [4]:
# Función para obtener los géneros de cada película y combinarlos en una cadena de texto. 
# Convierte los géneros de one-hot encoding a texto plano
def get_genres(row):
    genres = []
    for column in movies.columns[5:]:
        if row[column] == 1:
            genres.append(column)
    return ' '.join(genres)

# Aplica la función para crear la nueva columna 'genres'
movies['genres'] = movies.apply(get_genres, axis=1)

# Muestra el dataset resultante con la nueva columna 'genres' (eliminamos algunas columnas que no necesitamos)
movies = movies[['movie_id', 'title', 'release_date', 'video_release_date', 'imdb_url', 'genres']]
movies.head()

,movie_id,title,release_date,video_release_date,imdb_url,genres
0,1,Toy Story (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Toy%20Story%2...,animation children comedy
1,2,GoldenEye (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?GoldenEye%20(...,action adventure thriller
2,3,Four Rooms (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Four%20Rooms%...,thriller
3,4,Get Shorty (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Get%20Shorty%...,action comedy drama
4,5,Copycat (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Copycat%20(1995),crime drama thriller


Aplicamos la tecnica de TF-IDF, con el metodo TfidfVectorizer provisto por Scikit-Learn.

In [5]:
# Crea el vectorizador TF-IDF
tfidf_vectorizer = TfidfVectorizer()

# Crea la matriz TF-IDF para los géneros
tfidf_matrix = tfidf_vectorizer.fit_transform(movies['genres'])

Calculamos la similitud del coseno con la ayuda del metodo cosine_similarity de la librería Scikit-Learn

In [6]:
# Calcula la similitud de coseno entre todas las películas
cosine_sim = cosine_similarity(tfidf_matrix)

Creamos un metodo para obtener recomendaciones a partir del titulo de una pelicula (la cual asumimos que un usuario valoró positivamente)

In [7]:
# Función para obtener las películas más similares dado un título
def get_recommendations(title, cosine_sim=cosine_sim):
    idx = movies.index[movies['title'] == title].tolist()[0]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:11]
    movie_indices = [i[0] for i in sim_scores]
    return movies['title'].iloc[movie_indices]

# Ejemplo de uso
movie_title = 'Toy Story (1995)'
recommendations = get_recommendations(movie_title)
print(f"Recomendaciones para '{movie_title}':")
print(recommendations)

Recomendaciones para 'Toy Story (1995)':
421                Aladdin and the King of Thieves (1996)
101                                Aristocats, The (1970)
403                                      Pinocchio (1940)
624                        Sword in the Stone, The (1963)
945                         Fox and the Hound, The (1981)
968           Winnie the Pooh and the Blustery Day (1968)
1065                                         Balto (1995)
1077                              Oliver & Company (1988)
1408                            Swan Princess, The (1994)
1411    Land Before Time III: The Time of the Great Gi...
Name: title, dtype: object


Si observamos las recomendaciones generadas son del mismo genero que la pelicula objetivo.

**Ejercicio:**
La idea es que ahora ustedes creen un algoritmo de filtrado basado en contenido analizando el titulo de la misma (en vez del genero). Es decir, se espera recomendar peliculas cuyo titulo sea similar a los titulos de las peliculas elegidas por el usuario

In [8]:
tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix = tfidf_vectorizer.fit_transform(movies['title'])
cosine_sim = cosine_similarity(tfidf_matrix)

# Ejemplo de uso
movie_title = 'Toy Story (1995)'
recommendations = get_recommendations(movie_title)
print(f"Recomendaciones para '{movie_title}':")
print(recommendations)

Recomendaciones para 'Toy Story (1995)':
421                Aladdin and the King of Thieves (1996)
101                                Aristocats, The (1970)
403                                      Pinocchio (1940)
624                        Sword in the Stone, The (1963)
945                         Fox and the Hound, The (1981)
968           Winnie the Pooh and the Blustery Day (1968)
1065                                         Balto (1995)
1077                              Oliver & Company (1988)
1408                            Swan Princess, The (1994)
1411    Land Before Time III: The Time of the Great Gi...
Name: title, dtype: object


### Factorizacion de matrices

A continuación, se presenta un ejemplo de uso de la librería Surprise para generar recomendaciones a partir de la tecnica de Factorización de matrices.

La idea es que simplemente analicen el ejemplo paso por paso para entender como funciona este algoritmo.

**Nota:** Tener en cuenta que para la ejecución de este código es necesario tener instalada la librería Surprise

In [9]:
import pandas as pd
from surprise import Dataset, Reader
from surprise.model_selection import train_test_split
from surprise import SVD
from surprise import accuracy

# Cargar el archivo de calificaciones y películas
ratings = pd.read_csv('ml-100k/u.data', sep='\t', names=['user_id', 'movie_id', 'rating', 'timestamp'])
movies = pd.read_csv('ml-100k/u.item', sep='|', names=['movie_id', 'title', 'release_date', 'video_release_date', 'imdb_url',
                                                       'unknown', 'action', 'adventure', 'animation', 'children', 'comedy',
                                                       'crime', 'documentary', 'drama', 'fantasy', 'film_noir', 'horror',
                                                       'musical', 'mystery', 'romance', 'sci_fi', 'thriller', 'war', 'western'],
                     encoding='latin-1')

# Crear un objeto Reader para parsear los datos
reader = Reader(rating_scale=(1, 5))

# Cargar los datos en el formato esperado por Surprise
data = Dataset.load_from_df(ratings[['user_id', 'movie_id', 'rating']], reader)

# Dividir el conjunto de datos en conjuntos de entrenamiento y prueba
trainset, testset = train_test_split(data, test_size=0.2)

# Crear y entrenar el modelo de factorización de matrices (SVD)
model = SVD()
model.fit(trainset)

# Realizar predicciones en el conjunto de prueba
predictions = model.test(testset)

# Calcular y mostrar la precisión del modelo
accuracy.rmse(predictions)
print()

# Hacer predicciones para usuarios específicos
user_id = 1
item_id = 3
predicted_rating = model.predict(user_id, item_id).est
print(f"Predicción de la valoración para el usuario {user_id} y el elemento {item_id}: {predicted_rating}")
print()

# Obtener las recomendaciones para un usuario específico
# (por ejemplo, los elementos con las valoraciones más altas)
user_items = data.build_full_trainset().ur
user_items_ratings = [(item_rating[0], model.predict(user_id, item_id).est) for item_rating in user_items[user_id]]
user_items_ratings.sort(key=lambda x: x[1], reverse=True)

top_recommendations = user_items_ratings[:10]

print(f"Las 10 mejores recomendaciones para el usuario {user_id}:")
print()
for item_id, rating in top_recommendations:
    print(f"Elemento: {item_id}, Valoración: {rating}")


RMSE: 0.9407

Predicción de la valoración para el usuario 1 y el elemento 3: 3.382745250624616

Las 10 mejores recomendaciones para el usuario 1:

Elemento: 1, Valoración: 3.382745250624616
Elemento: 476, Valoración: 3.382745250624616
Elemento: 305, Valoración: 3.382745250624616
Elemento: 577, Valoración: 3.382745250624616
Elemento: 627, Valoración: 3.382745250624616
Elemento: 746, Valoración: 3.382745250624616
Elemento: 800, Valoración: 3.382745250624616
Elemento: 151, Valoración: 3.382745250624616
Elemento: 114, Valoración: 3.382745250624616
Elemento: 433, Valoración: 3.382745250624616
